# CNN MNIST

In [1]:
#importing functions from python3 to python2
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#importing numpy and tensorflow
import numpy as np
import tensorflow as tf

#ignore all the warnings and don't show them in the notebook
import warnings
warnings.filterwarnings('ignore')

#treshold on what messages are to be logged
tf.logging.set_verbosity(tf.logging.INFO)
#importing debug library
from tensorflow.python import debug as tf_debug

## Debugger

### Uncomment the below line and execute the code to run the debugger.

### Go to the link once you start execution    			http://localhost:6006/

In [2]:
#Uncomment the below line to run the debugger
#Add monitor=[hook] as a parameter to the estimators below
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
import wget

In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
            inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }
    #if predict mode, run the estimator and return the values
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [4]:
def main(unused_argv):
    # Load training data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images  # Returns np.array
    #load the the train data labels
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    #load the test data
    eval_data = mnist.test.images  # Returns np.array
    #load the test data labels
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)


In [5]:
#Run the model, steps set to 200 instead of 20000 as the execution time was large
#Changing steps back to 20000 in model training results in an accuracy of 97%
if __name__ == "__main__":
    tf.app.run()


Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_evaluation_master': '', '_train_distribute': None, '_master': '', '_save_checkpoints_secs': 600, '_num_ps_repl

INFO:tensorflow:loss = 2.2719352, step = 201
INFO:tensorflow:probabilities = [[0.09900723 0.11462514 0.09092633 0.10605785 0.09146865 0.09417575
  0.11214723 0.0945669  0.10088135 0.0961436 ]
 [0.08049084 0.09872862 0.09661689 0.11005732 0.10420978 0.09046147
  0.12652358 0.09144209 0.09536242 0.10610696]
 [0.10711027 0.09233667 0.09907777 0.1066141  0.08627798 0.09247319
  0.09784735 0.10574894 0.10252461 0.10998917]
 [0.08423807 0.11307806 0.09227416 0.10188986 0.10300422 0.09805868
  0.11163839 0.10074767 0.08932267 0.10574822]
 [0.09372701 0.12122765 0.08802908 0.09438772 0.09796006 0.09204914
  0.11708553 0.102095   0.10445922 0.08897963]
 [0.09116402 0.12388585 0.08721261 0.09776939 0.09717235 0.08646587
  0.10825744 0.10297467 0.10868313 0.09641473]
 [0.1007605  0.1075672  0.09587001 0.10164157 0.09865343 0.09514817
  0.10586936 0.1028494  0.09715656 0.09448386]
 [0.09469233 0.09751935 0.08371002 0.12727945 0.10725516 0.09109826
  0.10781467 0.08936065 0.11210964 0.08916048]
 [0

INFO:tensorflow:global_step/sec: 5.22099
INFO:tensorflow:probabilities = [[0.09825    0.09698748 0.08569925 0.11353071 0.100256   0.0939786
  0.11767921 0.09616213 0.09220389 0.10525275]
 [0.099778   0.11158741 0.0864189  0.09874582 0.09091851 0.10692114
  0.09881952 0.09668714 0.10644308 0.10368041]
 [0.08823047 0.08984425 0.09948711 0.12422841 0.10085653 0.1021429
  0.09239664 0.09011306 0.08918258 0.12351801]
 [0.09766591 0.1006669  0.10244506 0.11565657 0.08924171 0.09361098
  0.10904538 0.0893084  0.10211539 0.10024365]
 [0.09534667 0.10655404 0.10068792 0.08936308 0.08811914 0.0888257
  0.11008744 0.09671874 0.12556686 0.09873036]
 [0.09578708 0.0963055  0.09245717 0.10582525 0.10552511 0.09632697
  0.11722723 0.09080406 0.10630462 0.09343704]
 [0.13145925 0.09018727 0.09124753 0.10991424 0.09849955 0.09408844
  0.0937199  0.08484497 0.11187895 0.09415994]
 [0.09495308 0.09165888 0.09874056 0.10056648 0.08045882 0.10218057
  0.10615581 0.10039683 0.11246722 0.11242171]
 [0.120723

INFO:tensorflow:loss = 2.251534, step = 301 (19.155 sec)
INFO:tensorflow:probabilities = [[0.1119133  0.08083475 0.09050848 0.10049442 0.09710395 0.10136689
  0.10485674 0.10385396 0.11265875 0.09640875]
 [0.12033607 0.08858202 0.0935526  0.12032652 0.09716502 0.1014812
  0.09702037 0.07993561 0.10230549 0.09929509]
 [0.09516314 0.08599374 0.09388835 0.10554901 0.0943273  0.10793874
  0.12207069 0.08649519 0.10162334 0.1069505 ]
 [0.10483898 0.08647919 0.09109266 0.12867755 0.08897209 0.09284335
  0.10283794 0.0931694  0.10903975 0.10204913]
 [0.11320969 0.10535973 0.09128634 0.12202836 0.09899772 0.10030694
  0.09276367 0.0959414  0.08621871 0.0938875 ]
 [0.10804955 0.07383121 0.09950979 0.09777597 0.10134386 0.10474621
  0.11119161 0.09112078 0.10671493 0.10571606]
 [0.09411892 0.09961388 0.09090859 0.11484211 0.10861212 0.08765401
  0.10555055 0.08867428 0.10194515 0.10808045]
 [0.10241337 0.11955831 0.09681447 0.09894878 0.10064323 0.09391966
  0.09649049 0.09227607 0.10008215 0.09

INFO:tensorflow:Saving checkpoints for 400 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 2.208863.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-20-18:47:02
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-20-18:47:08
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.4699, global_step = 400, loss = 2.2138305
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 400: /tmp/mnist_convnet_model/model.ckpt-400
{'loss': 2.2138305, 'global_step': 400, 'accuracy': 0.4699}


SystemExit: 